# Case 1 說明
只對第一個 user input 執行 RAG

Retrieve flow and related QA

In [1]:
import os
import time
from openai import OpenAI
from tabulate import tabulate
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

In [2]:
import pandas as pd
import numpy as np
import faiss

csv_file = 'data\embeddingsV3_3072.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

C:\Users\USER\AppData\Local\Temp\ipykernel_47804\2711399014.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
def get_similar_qa(input):
    response = client.embeddings.create(input=input, model="text-embedding-3-large", dimensions=3072)
    question = np.array(response.data[0].embedding, ndmin=2)
    k = 3
    distances, indices = index.search(question, k)
    csv_question = 'data\manual.csv'
    qdf = pd.read_csv(csv_question)
    qarray = qdf.to_numpy()
    answer = [(qarray[i], float(dist)) for dist, i in zip(distances[0], indices[0])]
    history = ""
    min_distance = 1
    Qcount = 0
    for i in range(0,k):
        if answer[i][1]<min_distance:
            Qcount+=1
            history = history +str(i+1)+". "+ answer[i][0][0]+"\n"+answer[i][0][1]+"\n"
    return Qcount, history

def get_similar_flow(input):
    pass

# Instruction

In [4]:
instructions = [
    "1. You are Flora customer service chat bot. You are prohibited replying unrelated to Flora",
    "2. When user ask a question, you must follow the flow, resolving the problem one step at a time.",
    "3. If the user asks another question before the previous question is solved, deal with the new question first and then go back to the steps of the previous question for continued processing.",
    "4. When you encounter question concerning email, you must check email fitting RFC5322 standard or not. if not, ask again"
]

instruction = ""
for i in instructions:
    instruction += i

print(instruction)

1. You are Flora customer service chat bot. You are prohibited replying unrelated to Flora2. When user ask a question, you must follow the flow, resolving the problem one step at a time.3. If the user asks another question before the previous question is solved, deal with the new question first and then go back to the steps of the previous question for continued processing.4. When you encounter question concerning email, you must check email fitting RFC5322 standard or not. if not, ask again


In [5]:
assistant = client.beta.assistants.create(
    name="Flora's Customer Service",
    instructions=instruction,
    model="gpt-3.5-turbo-1106"
    # tools = [
    #     {
    #         "type": "function",
    #         "function": {
    #             "name": "get_history_question_and_answer",
    #             "description": "Get 1 to 3 history questions and answers that is similar to current question",
    #             "parameters": {
    #                 "type": "object",
    #                 "properties": {
    #                     "current_question": {
    #                         "type": "string",
    #                         "description": "user's question"
    #                     }
    #                 },
    #                 "required": ["current_question"]
    #             }
    #         }
    #     }
    # ]
)

thread = client.beta.threads.create()

# Flow

In [6]:
# flow = [
#     "Follow the steps to answer questions, if another question comes out, deal with the current problem first, then go back to the step of previous issue if user ask again:",
#     "Ask the user for their email.",
#     "Ask the user for the time when the tree was killed.",
#     "The last step, you must respond based on the reference answer of the user's initial question if history questions and reference answers are provided. If not, you must reply only a special mark:'*'."
# ]

'''
ask user for their email;
if(got user email){
    ask user for the time when the tree was killed;
    if(got the time){
        respond based on the reference answer of the user's initial question if history questions and reference answers are provided. If not, you must reply only a special mark:'*'.
    }
    else{
        ask the time again.
    }
}
else{ask email again}
'''

flow = "ask user for their email;if(got user email){ask user for the time when the tree was killed;if(got the time){respond based on the reference answer of the user's initial question if history questions and reference answers are provided. If not, you must reply only a special mark:'*'.}else{ask the time again.}}else{ask email again}"

In [7]:
user_input = "I accidentally killed my tree. Is there any way to restore it?"

In [8]:
# prompt = get_similar_flow(user_input) + get_similar_qa(user_input) + user_input

prompt = ""

# for i in flow:
#     prompt = prompt + i + "\n"

prompt += flow

similar_qa = get_similar_qa(user_input)
similar_flow = get_similar_flow(user_input)

if similar_qa[0] > 0:
    prompt += "\nHere is " + str(similar_qa[0]) + " history questions and answers related to this question according to Flora's database:\n" + similar_qa[1] + "\n"

prompt = prompt + "here is the user input \n" + user_input
print(prompt)

ask user for their email;if(got user email){ask user for the time when the tree was killed;if(got the time){respond based on the reference answer of the user's initial question if history questions and reference answers are provided. If not, you must reply only a special mark:'*'.}else{ask the time again.}}else{ask email again}
Here is 3 history questions and answers related to this question according to Flora's database:
1.  I just started using this app and was looking around trying to figure the features out and accidentally killed a tree. I don'twant that to be there permanently. Is there anything I can do?
We currently don't support the function of deleting a story as we believe every record matters. If you get a dead tree because of the system error, you could inform us by reporting 'False Detection'(the ! icon on the top right corner on the tree dead page) or through this feedback system. We would correct the result for you.
2. What's the best way to ensure my tree doesn't get k

# 2nd round

In [15]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content = prompt
    #content = "Iambigashell.gg.ez"
    #content = "It's around 4:00 PM on March 28th."
)

In [16]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [17]:
while run.status == "queued" or run.status == "in_progress":
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
print(run.status)
# print(tabulate(run,headers=['run','content'],tablefmt='simple_outline'))

completed


In [18]:
time.sleep(5)
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order='asc'
)

In [19]:
for i in messages.data[-5:]:
    print(i.content[0].text.value)
    print("==========")

ask user for their email;if(got user email){ask user for the time when the tree was killed;if(got the time){respond based on the reference answer of the user's initial question if history questions and reference answers are provided. If not, you must reply only a special mark:'*'.}else{ask the time again.}}else{ask email again}
Here is 3 history questions and answers related to this question according to Flora's database:
1.  I just started using this app and was looking around trying to figure the features out and accidentally killed a tree. I don'twant that to be there permanently. Is there anything I can do?
We currently don't support the function of deleting a story as we believe every record matters. If you get a dead tree because of the system error, you could inform us by reporting 'False Detection'(the ! icon on the top right corner on the tree dead page) or through this feedback system. We would correct the result for you.
2. What's the best way to ensure my tree doesn't get k

# Delete Assistant

In [21]:
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_TSaRSWPCoIhNlQ884xji7KRD', deleted=True, object='assistant.deleted')